In [2]:
import json
import numpy as np

In [3]:
class Constraint:

    def __init__(self, data) -> None:
        super().__init__()
        self.coefs = data['coefs']
        self.b = data['b']
        self.type = data['type']
        if self.type == 'eq':
            self.function = lambda values: np.sum(np.multiply(self.coefs, values)) == self.b
        if self.type == 'leq':
            self.function = lambda values: np.sum(np.multiply(self.coefs, values)) <= self.b
        if self.type == 'geq':
            self.function = lambda values: np.sum(np.multiply(self.coefs, values)) >= self.b


    def check(self, values):
        return self.function(values)


class Task:

    def __init__(self, path) -> None:
        super().__init__()
        data = json.loads(open(path).read())
        self.f = data['f']
        self.goal = data['goal']
        self.constraints = []
        self.was_goal_changed = False
        self.first_step_f = np.zeros(len(self.f))
        self.artificial_indexes = []
        for constraint in data['constraints']:
            self.constraints.append(Constraint(constraint))


    def get_column(self, x):
        column = []
        for i in self.constraints:
            column.append(i.coefs[x])
        return column


    def check_constraints(self, values):
        flag = True
        for constraint in self.constraints:
            flag &= constraint.check(values)

        return flag


    def evaluate(self, values):
        return np.sum(np.multiply(self.f, values))

In [4]:
class SimplexTable():

    def __init__(self, task) -> None:
        self.m = len(task.constraints) + 1
        self.n = len(task.f) + 1
        self.table = np.zeros([self.m, self.n])
        self.blocked_columns = task.artificial_indexes
        self.basis = []
        self.second_fase_f = task.f
        for i in range(self.m):
            if i == self.m - 1:
                self.table[i, 0] = 0
                for j in range(1, self.n):
                    self.table[i, j] = task.first_step_f[j - 1]
            else:
                for j in range(self.n):
                    if j == 0:
                        self.table[i, j] = task.constraints[i].b
                    else:
                        self.table[i, j] = task.constraints[i].coefs[j - 1]
            
            if i < self.m - 1:
                need_basis_column = True
                for j in range(1, self.n):
                    tmp = task.get_column(j - 1)
                    if tmp[i] == 1 and tmp.count(0) == self.m - 2 and need_basis_column:
                        need_basis_column = False
                        self.basis.append(j - 1)
                  

    def first_phase(self):
        first_iter = True
        if len(self.blocked_columns) == 0:
            return self.table
        
        while self.end(first_iter):
            main_column = self.find_column(True, first_iter)
            main_row = self.find_row(main_column)

            self.basis[main_row] = main_column - 1
            new_table = np.zeros([self.m, self.n])

            for i in range(self.n):
                new_table[main_row, i] = self.table[main_row, i] / self.table[main_row, main_column]

            for i in range(self.m):
                if i == main_row:
                    continue

                for j in range(self.n):
                    new_table[i, j] = self.table[i, j] - self.table[i, main_column] * new_table[main_row, j]

            self.table = new_table

            if first_iter:
                first_iter = False

        return self.table


    def second_phase(self):
        self.next_phase()
        while self.end(True):
            main_column = self.find_column(False, True)
            main_row = self.find_row(main_column)
            
            if main_column - 1 == self.basis[main_row]:
                break
            
            self.basis[main_row] = main_column - 1
            new_table = np.zeros([self.m, self.n])

            for i in range(self.n):
                new_table[main_row, i] = self.table[main_row, i] / self.table[main_row, main_column]

            for i in range(self.m):
                if i == main_row:
                    continue

                for j in range(self.n):
                    new_table[i, j] = self.table[i, j] - self.table[i, main_column] * new_table[main_row, j]
                
            new_table[self.m - 1, 0] = 0
            for i in range(self.m - 1):
                new_table[self.m - 1, 0] += new_table[i, 0] * self.second_fase_f[self.basis[i]]

            self.table = new_table

        return self.table


    def next_phase(self):
        for i in range(1, self.n):
            self.table[self.m - 1, i] = 0

        for i in range(1, self.n):
            if self.second_fase_f[i - 1] != 0:
                if not i - 1 in self.basis:
                    self.table[self.m - 1, i] += self.second_fase_f[i - 1]
                else:
                    for j in range(1, self.n):
                        if j != i and self.table[self.basis.index(i - 1), j] != 0:
                            self.table[self.m - 1, j] -= self.second_fase_f[i - 1] * self.table[self.basis.index(i - 1), j]

        self.table[self.m - 1, 0] = 0
        for i in range(self.m - 1):
            self.table[self.m - 1, 0] += self.table[i, 0] * self.second_fase_f[self.basis[i]]


    def end(self, search_max):
        flag = False
        if search_max:
            for i in range(1, self.n):
                if self.table[self.m - 1, i] > 0:
                    flag = True
                    break
        else:
            for i in range(1, self.n):
                if self.table[self.m - 1, i] < 0:
                    flag = True
                    break
        
        return flag


    def find_column(self, first_phase, search_max):
        for i in range(1, self.n):
            if first_phase or i - 1 not in self.blocked_columns:
                main_column = i
                break

        if search_max:
            for i in range(main_column, self.n):
                if (first_phase or i - 1 not in self.blocked_columns) and self.table[self.m - 1, i] > self.table[self.m - 1, main_column]:
                    main_column = i
        else:
            for i in range(main_column, self.n):
                if (first_phase or i - 1 not in self.blocked_columns) and self.table[self.m - 1, i] < self.table[self.m - 1, main_column]:
                    print(main_column, i)                    
                    main_column = i

        return main_column


    def find_row(self, main_column):
        main_row = 0
        for i in range(self.m - 1):
            if self.table[i, main_column] > 0:
                main_row = i
                break

        for i in range(main_row + 1, self.m - 1):
            if self.table[i, main_column] > 0 and (self.table[i, 0] / self.table[i, main_column] < self.table[main_row, 0] / self.table[main_row, main_column]):
                main_row = i

        return main_row
        

In [5]:
def make_canonical_form(task):
    if task.goal == "min":
        task.f = np.multiply(task.f, -1)
        task.goal = "max"

    auxiliary_ind = 0
    m = len(task.constraints)
    n = len(task.f)

    for constraint in task.constraints:
        if constraint.type == "geq":
            constraint.coefs = np.concatenate([constraint.coefs, np.zeros(auxiliary_ind), [-1]])
            constraint.function = lambda values: np.sum(np.multiply(constraint.coefs, values)) == constraint.b
            constraint.type = "eq"
            auxiliary_ind += 1
        if constraint.type == "leq":
            constraint.coefs = np.concatenate([constraint.coefs, np.zeros(auxiliary_ind), [1]])
            constraint.function = lambda values: np.sum(np.multiply(constraint.coefs, values)) == constraint.b
            constraint.type = "eq"
            auxiliary_ind += 1

    for constraint in task.constraints:
        constraint.coefs = np.concatenate([constraint.coefs, np.zeros(auxiliary_ind+n-len(constraint.coefs))])
        if constraint.b < 0:
            constraint.coefs = np.multiply(constraint.coefs, -1)
            constraint.b = np.multiply(constraint.b, -1)
    
    n = len(task.f) + auxiliary_ind
    task.first_step_f = np.concatenate([task.first_step_f, np.zeros(auxiliary_ind)])
    
    for i in range(m):
        need_basis_column = True
        for j in range(n):
            tmp = task.get_column(j)
            if tmp[i] == 1 and tmp.count(0) == m - 1 and need_basis_column:
                need_basis_column = False
        if need_basis_column:
            for j in range(m):
                if j == i:
                    task.constraints[j].coefs = np.concatenate([task.constraints[j].coefs, [1]])
                else:
                    task.constraints[j].coefs = np.concatenate([task.constraints[j].coefs, [0]])
                constraint.function = lambda values: np.sum(np.multiply(constraint.coefs, values)) == constraint.b
            
            task.first_step_f = np.concatenate([task.first_step_f, [1]])
            task.artificial_indexes.append(len(task.f) + auxiliary_ind)
            auxiliary_ind += 1

    task.f = np.concatenate([task.f, np.zeros(auxiliary_ind)])

In [6]:
task = Task('example5.json')
make_canonical_form(task)

simplex_table = SimplexTable(task)
print(simplex_table.blocked_columns)
print(simplex_table.basis)
print(simplex_table.table)
print("__________________")
simplex_table.first_phase()
print(simplex_table.basis)
print(simplex_table.table)
print("__________________")
simplex_table.second_phase()
print(simplex_table.basis)
print(simplex_table.table)

#28/5, 0, 0, 1/5, 12/5

[5]
[0, 5, 2]
[[ 1.  1.  0.  0.  1. -2.  0.]
 [ 2.  0.  0.  0. -2.  1.  1.]
 [ 3.  0.  0.  1.  3.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.]]
__________________
1 5
[0, 4, 2]
[[ 5.  1.  0.  0. -3.  0.  2.]
 [ 2.  0.  0.  0. -2.  1.  1.]
 [ 1.  0.  0.  1.  5.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.  1.]]
__________________
[0, 4, 3]
[[ 5.6  1.   0.   0.6  0.   0.   1.4]
 [ 2.4  0.   0.   0.4  0.   1.   0.6]
 [ 0.2  0.   0.   0.2  1.   0.  -0.2]
 [ 2.2  0.   0.  -0.2  0.   0.  -0.8]]
